# Communication demo

## Load keys

In [2]:
import pandas as pd
import numpy as np

# numberのkeyからboolの配列に変換する関数
def num2bool_key(num,key_len):
    key = np.zeros(key_len,dtype=bool)
    for i in range(key_len):
        np.mod(num,2) == 0
        if np.mod(num,2) == 0:
            key[i] = False
        else:
            key[i] = True
        num = num//2

    return key


key_df = pd.read_csv('./communication_key_info/pair3_length8.csv')
use_num_of_key = 16
key1_num = key_df.loc[use_num_of_key,['base1']].values
key2_num = key_df.loc[use_num_of_key,['base2']].values
key3_num = key_df.loc[use_num_of_key,['base3']].values

key1 = num2bool_key(key1_num,8)
key2 = num2bool_key(key2_num,8)
key3 = num2bool_key(key3_num,8)

print(f'key1:{key1}')
print(f'key2:{key2}')
print(f'key3:{key3}')

key1:[ True  True False False  True False False False]
key2:[ True False  True False  True False False False]
key3:[ True False False  True  True False False False]


## Make send data

In [3]:
send_data1 = np.array([1,0,0,1,1,1,1],dtype=bool)
send_data2 = np.array([0,1,0,1,1,0,1],dtype=bool)
send_data3 = np.array([1,0,1,1,1,0,1],dtype=bool)

## Modulate data

In [4]:
from bedcmm.communication import *

key1_mod_data = modulate(send_data1,key1)
key2_mod_data = modulate(send_data2,key2)
key3_mod_data = modulate(send_data3,key3)

print(f'modulated data1:{key1_mod_data}')
print(f'modulated data2:{key2_mod_data}')
print(f'modulated data3:{key3_mod_data}')

modulated data1:[ True  True False False  True False False False False False False False
 False False False False False False False False False False False False
  True  True False False  True False False False  True  True False False
  True False False False  True  True False False  True False False False
  True  True False False  True False False False]
modulated data2:[False False False False False False False False  True False  True False
  True False False False False False False False False False False False
  True False  True False  True False False False  True False  True False
  True False False False False False False False False False False False
  True False  True False  True False False False]
modulated data3:[ True False False  True  True False False False False False False False
 False False False False  True False False  True  True False False False
  True False False  True  True False False False  True False False  True
  True False False False False False False False 

## multiplexing data

In [5]:
all_mod_data = np.array([key1_mod_data,key2_mod_data,key3_mod_data],dtype=bool)
send_signal = multiplexing(all_mod_data)

print(f'send signal:{send_signal}')

send signal:[ True  True False  True  True False False False  True False  True False
  True False False False  True False False  True  True False False False
  True  True  True  True  True False False False  True  True  True  True
  True False False False  True  True False False  True False False False
  True  True  True  True  True False False False]


## Demodulate Data

In [6]:
# all pattern
key1_demod_signal = demodulate(send_signal,key1)
key2_demod_signal = demodulate(send_signal,key2)
key3_demod_signal = demodulate(send_signal,key3)

filt = np.ones(len(key1))
filtered_data1 = np.convolve(key1_demod_signal,filt,mode='same')
demod_data1 = np.array(filtered_data1[::len(key1)],dtype=bool)
filtered_data2 = np.convolve(key2_demod_signal,filt,mode='same')
demod_data2 = np.array(filtered_data2[::len(key1)],dtype=bool)
filtered_data3 = np.convolve(key3_demod_signal,filt,mode='same')
demod_data3 = np.array(filtered_data3[::len(key1)],dtype=bool)

print(f'send  data1:{send_data1}')
print(f'demod data1:{demod_data1}')
print(f'send  data2:{send_data2}')
print(f'demod data2:{demod_data2}')
print(f'send  data3:{send_data3}')
print(f'demod data3:{demod_data3}')

send  data1:[ True False False  True  True  True  True]
demod data1:[ True False False  True  True  True  True]
send  data2:[False  True False  True  True False  True]
demod data2:[False  True False  True  True False  True]
send  data3:[ True False  True  True  True False  True]
demod data3:[ True False  True  True  True False  True]


In [7]:
# one by one pattern

l_o_esult1 = []
l_o_esult2 = []
l_o_esult3 = []

for i in range(len(send_signal)-len(key1)+1):
    calc_data = send_signal[i:i+len(key1)]
    l_o_esult1.append(demodulate_logical_operations(calc_data,key1))
    l_o_esult2.append(demodulate_logical_operations(calc_data,key2))
    l_o_esult3.append(demodulate_logical_operations(calc_data,key3))

filt = np.ones(len(key1))
filtered_data1 = np.convolve(l_o_esult1,filt,mode='same')
l_o_esult1 = np.array(filtered_data1[::len(key1)],dtype=bool)
filtered_data2 = np.convolve(l_o_esult2,filt,mode='same')
l_o_esult2 = np.array(filtered_data2[::len(key1)],dtype=bool)
filtered_data3 = np.convolve(l_o_esult3,filt,mode='same')
l_o_esult3 = np.array(filtered_data3[::len(key1)],dtype=bool)

print(f'send     data1:{send_data1}')
print(f'lo demod data1:{l_o_esult1}')
print(f'send     data2:{send_data2}')
print(f'lo demod data2:{l_o_esult2}')
print(f'send     data3:{send_data3}')
print(f'lo demod data3:{l_o_esult3}')


send     data1:[ True False False  True  True  True  True]
lo demod data1:[ True False False  True  True  True  True]
send     data2:[False  True False  True  True False  True]
lo demod data2:[False  True False  True  True False  True]
send     data3:[ True False  True  True  True False  True]
lo demod data3:[ True False  True  True  True False  True]
